In [1]:
### Python Libraries ===========================
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

from numpy import sqrt
from numpy.random import randn
from numpy.linalg import cholesky as chol
### ============================================



nMC = 10_000 # Number of Monte Carlo iterations

# Storage matrices for estimates
TRUE   = []
OLS    = []
BIASED = []

for iMC in trange(nMC):  #### Start Monte Carlo iterations here
    
    #### ========================| GENERATE ARTIFICIAL DATA |=================
    # Settings for Monte Carlo exercise
    n = 100    # Number of data observations
    p = 3      # Number of predictors

    # Generate regressors that might be correlated
    # first generate correlation matrix
    rho = 0.99    # Correlation coefficient (keep between -1 and 1)
    R   = np.zeros((p,p))   # Space in memory for correlation matrix
    for i in range(p):
        for j in range(p):
            R[i, j] = rho ** (abs(i-j))

    # next generate correlated predictors from a Normal distribution
    X = randn(n,p) @ chol(R)

    # generate regression coefficients. Here you can either set each parameter
    # yourself, e.g. beta = [1.2, -0.8] in the case of p=2 predictors, beta =
    # [2.1, -1.4, 3.6] in the case of p=3 predictors, etc. Here below I specify
    # a beta vector that complies with p=2. If you do not want to input each
    # value of beta every time (e.g. because you want to try p=20), then you
    # can generate this randomly (using a command of the form beta =
    # rand(p,1))
    beta = np.array([1.3, 0.9, -0.9])

    # Generate regression data y
    sigma2 = 1    # Regression variance
    y = (X @ beta).reshape((-1, 1)) + sqrt(sigma2) * randn(n,1)   # These are my data y following the regression data generating process
    ##### =====================================================================


    # From here on, you have in your memory data X and y, and you can treat
    # them as a sample from a population. The only difference is that you now
    # know that y DOES come from a linear regression model with p predictors.
    # You can do various experiments. For example, if you use all p predictors
    # you can estimate the regression model with various values of n
    # (observations) and check how OLS becomes more precise as n increases. The
    # experiment we are going to do here is that of omitted variable bias.
    # Assume that you are given p predictors in X but you only use the first
    # one
    X_est = X[:,[0]]

    # This means that OLS estimates in the TRUE model (the one in the data
    # generating process above) is:
    beta_OLS  = np.linalg.inv(X.T @ X) @ (X.T @ y)

    # ...while OLS estimates in our misspecified model with only one predictor
    # are
    beta_OLS_omitbias = np.linalg.inv(X_est.T @ X_est) @ (X_est.T @ y)

    # The true parameters are known of course with precision, and are provided
    # from the DGP above
    beta_TRUE = beta

    # Save results from i-th Monte Carlo iteration
    # These are p estimates for each of the nMC datasets
    TRUE  .append(beta_TRUE                                       .tolist())
    OLS   .append(beta_OLS.reshape(-1)                            .tolist())
    BIASED.append(np.append(beta_OLS_omitbias, np.zeros((p-1, 1))).tolist())

print(pd.DataFrame(
    np.array([TRUE, OLS, BIASED]).mean(axis = 1).T,
    columns = ['TRUE', 'OLS', 'BIASED'],
    index   = [f'beta_{i}' for i in range(p)]
))

100%|██████████| 10000/10000 [00:00<00:00, 12252.92it/s]

        TRUE       OLS    BIASED
beta_0   1.3  1.302047  1.342495
beta_1   0.9  0.893718  0.000000
beta_2  -0.9 -0.925110  0.000000
